# Human in the Loop

There are many occasions where you would like to put a human in the loop to keep tabs on what an agent is doing
.


# Setup

In [1]:
from dotenv import load_dotenv

In [2]:
_ = load_dotenv()

# Global Imports

In [3]:
import operator
from typing import Annotated, TypedDict

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import END, StateGraph

In [4]:
memory = SqliteSaver.from_conn_string(":memory:")

> **We're gonna perform a small modification**.

In the previous examples, we annotated the messages list with the `operator.add`. That added messages to the existing messages array. 

However, for this human-in-the-loop interactions, **we may want to actually replace these existing messages**.

In order to do that, we're going to write a custom `reduce_messages` function, that basically looks for messages with the same id, and appends them otherwise.

In [5]:
from uuid import uuid4

from langchain_core.messages import AIMessage

In [7]:
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # Assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # Merge the new messages (from the right) to the existing ones (from the left)
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # Replace any existing message (from the left)
            # By one with the same id (from the right)
            if existing.id == message.id:
                merged[i] = message
                break
        # This `else` clause suggests no `break` was encountered, it's not mandatory
        else:
            # Append any new message to the end
            merged.append(message)
    return merged

In [8]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [9]:
tool = TavilySearchResults(max_results=2)

# Manual Human Approval

We'll implement the same Agent, but **with one small crucial modification**.

In `graph.compile`, we will set a new parameter `interrupt_before=["action"]` to **allow manual approval before running actions**.

In [11]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]  # 👈 Allow manual human approval before running tools
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

> **NOTE**
>
> Sometimes, you may want to interrupt only before a certain tool is called. This is covered in other parts of the documentation. **FIND LINK TO THIS!**

Let's initialize the Agent exactly like before.

In [12]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)